# Setup

In [ ]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("Opowiedz jakiś żart na temat {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

# Invoke

## without LCEL

In [23]:
from typing import List

import openai


prompt_template = "Opowiedz jakiś żart na temat  {topic}"

client = openai.OpenAI()

def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

invoke_chain("lody")

'Dlaczego lody zawsze są wesołe?\n\nBo zawsze mają lizaka!'

## LCEL

In [24]:
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template( "Opowiedz jakiś żart na temat  {topic}" )

model = ChatOpenAI(model="gpt-3.5-turbo")

output_parser = StrOutputParser()

chain = {"topic": RunnablePassthrough()} | prompt | model | output_parser

chain.invoke("lody")

'Dlaczego lody są tak zimne?\n\nBo mają dużo lodu w sobie!'

# Stream

In [7]:
import time

## without LCEL

In [8]:
from typing import Iterator


def stream_chat_model(messages: List[dict]) -> Iterator[str]:
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        stream=True,
    )
    for response in stream:
        content = response.choices[0].delta.content
        if content is not None:
            yield content

def stream_chain(topic: str) -> Iterator[str]:
    prompt_value = prompt.format(topic=topic)
    return stream_chat_model([{"role": "user", "content": prompt_value}])


for chunk in stream_chain("ice cream"):
    time.sleep(0.2)
    print(chunk, end="", flush=True)

Why did the ice cream truck break down?
Because it had too many frozen treats!

## LCEL

In [10]:
for chunk in chain.stream("ice cream"):
    time.sleep(0.2)
    print(chunk, end="", flush=True)

Why did the ice cream truck break down? 

Because it had too many sundae drivers!

# Batch

## without LCEL

In [25]:
from concurrent.futures import ThreadPoolExecutor


def batch_chain(topics: list) -> list:
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(executor.map(invoke_chain, topics))

batch_chain(["lody", "spaghetti", "pierogi"])

['Dlaczego lody są najlepszymi słuchawkami?\n\nBo są zawsze mrożone!',
 'Dlaczego spaghetti jest świetnym jedzeniem dla muzyków?\n\nBo potrafią grać na widelcach!',
 'Dlaczego pierogi nie mogą grać w karty?\n\nBo zawsze trzymają wszystkie asy w sobie!']

## LCEL

In [15]:
chain.batch(["ice cream", "spaghetti", "dumplings"])

KeyboardInterrupt: 

# Async

## without LCEL

In [16]:
async_client = openai.AsyncOpenAI()

async def acall_chat_model(messages: List[dict]) -> str:
    response = await async_client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

async def ainvoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return await acall_chat_model(messages)

In [17]:
await ainvoke_chain("ice cream")

'Why did the ice cream truck break down?\n\nBecause it had too many "scoops" of ice cream!'

## LCEL

In [18]:
chain.ainvoke("ice cream")

<coroutine object RunnableSequence.ainvoke at 0x00000205D24966C0>

# LLM instead of chat model

## without LCEL

In [21]:
def call_llm(prompt_value: str) -> str:
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt_value,
    )
    return response.choices[0].text

def invoke_llm_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    return call_llm(prompt_value)

invoke_llm_chain("ice cream")

'\n\nWhy did the ice cream go to therapy?\n\nBecause it was feeling a little'

## LCEL

In [20]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")

llm_chain = {"topic": RunnablePassthrough()} | prompt | llm | output_parser

llm_chain.invoke("ice cream")

'\n\nWhy did the ice cream go to therapy? Because it was feeling a little melon-coly.'

# Different model provider

In [27]:
%pip install anthropic

  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/846.4 kB ? eta -:--:--
   - ------------------------------------- 30.7/846.4 kB 660.6 kB/s eta 0:00:02
   -------- ------------------------------- 174.1/846.4 kB 2.1 MB/s eta 0:00:01
   ------------------------- -------------- 532.5/846.4 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 846.4/846.4 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ----------------- ---------------------- 0.9/2.2 MB 30.1 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 28.0 MB/s eta 0:00:00
Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
   ---------------------------------------- 0.0/170.9 kB ? eta -:--:--
   ---------------------------------------- 170.9

## Without LCEL

In [28]:
import anthropic

anthropic_template = f"Human:\n\n{prompt_template}\n\nAssistant:"
anthropic_client = anthropic.Anthropic()

def call_anthropic(prompt_value: str) -> str:
    response = anthropic_client.completions.create(
        model="claude-2",
        prompt=prompt_value,
        max_tokens_to_sample=256,
    )
    return response.completion    

def invoke_anthropic_chain(topic: str) -> str:
    prompt_value = anthropic_template.format(topic=topic)
    return call_anthropic(prompt_value)

invoke_anthropic_chain("lody")

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

## LCEL

In [33]:
from langchain_community.chat_models import ChatAnthropic

anthropic = ChatAnthropic(model="claude-2")

anthropic_chain = {"topic": RunnablePassthrough()} | prompt  | anthropic | output_parser

anthropic_chain.invoke("ice cream")

ValidationError: 1 validation error for ChatAnthropic
__root__
  Did not find anthropic_api_key, please add an environment variable `ANTHROPIC_API_KEY` which contains it, or pass `anthropic_api_key` as a named parameter. (type=value_error)

# Runtime configurability

## Without LCEL


In [ ]:
def invoke_configurable_chain(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> str:
    if model == "chat_openai":
        return invoke_chain(topic)
    elif model == "openai":
        return invoke_llm_chain(topic)
    elif model == "anthropic":
        return invoke_anthropic_chain(topic)
    else:
        raise ValueError(
            f"Received invalid model '{model}'."
            " Expected one of chat_openai, openai, anthropic"
        )

def stream_configurable_chain(
    topic: str, 
    *, 
    model: str = "chat_openai"
) -> Iterator[str]:
    if model == "chat_openai":
        return stream_chain(topic)
    elif model == "openai":
        # Note we haven't implemented this yet.
        return stream_llm_chain(topic)
    elif model == "anthropic":
        # Note we haven't implemented this yet
        return stream_anthropic_chain(topic)
    else:
        raise ValueError(
            f"Received invalid model '{model}'."
            " Expected one of chat_openai, openai, anthropic"
        )

def batch_configurable_chain(
    topics: List[str], 
    *, 
    model: str = "chat_openai"
) -> List[str]:
    # You get the idea
    ...

async def abatch_configurable_chain(
    topics: List[str], 
    *, 
    model: str = "chat_openai"
) -> List[str]:
    ...

invoke_configurable_chain("ice cream", model="openai")
stream = stream_configurable_chain(
    "ice_cream", 
    model="anthropic"
)
for chunk in stream:
    print(chunk, end="", flush=True)

# batch_configurable_chain(["ice cream", "spaghetti", "dumplings"])
# await ainvoke_configurable_chain("ice cream")

## LCEL

In [34]:
from langchain_core.runnables import ConfigurableField


configurable_model = model.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="chat_openai", 
    openai=llm,
    anthropic=llm, # small cheating since don;t have key for instantiate antropic
)

configurable_chain = {"topic": RunnablePassthrough()} | prompt | configurable_model | output_parser

In [36]:
model1_resp = configurable_chain.invoke(
    "ice cream", 
    config={"model": "openai"}
)

print(model1_resp)

model2_stream = configurable_chain.stream(
    "ice cream", 
    config={"model": "anthropic"}
)

for chunk in model2_stream:
    time.sleep(0.2)
    print(chunk, end="", flush=True)

#configurable_chain.batch(["ice cream", "spaghetti", "dumplings"])

# await configurable_chain.ainvoke("ice cream")

Dlaczego lody są zawsze gotowe do imprezy?

Bo zawsze mają w zamrażarce swoje "kubki"!
Dlaczego lody nie lubią iść na imprezy?

Bo się zawsze rozpuszczają!